# Test of the clasification pipeline

In [1]:
import os

# Function to change to the parent directory
def change_to_parent_directory():
    # Check if the directory has already been changed
    if not os.environ.get('DIR_CHANGED'):
        try:
            current_dir = os.path.dirname(os.path.abspath(__file__))
        except NameError:
            current_dir = os.getcwd()
        parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
        os.chdir(parent_dir)
        os.environ['DIR_CHANGED'] = '1'
        print(f"Current working directory changed to: {os.getcwd()}")
    else:
        print("Directory has already been changed.")

# Call the function to change the working directory
change_to_parent_directory()

Current working directory changed to: /home/polonipeter/praca/praca/radekRepo/clf/domainradar-clf


## Optional: Create testing dataset
Note: If you want do to this, set create_test_parquet to **True**

In [2]:
create_test_parquet = True

if create_test_parquet:
    import pandas as pd
    import numpy as np

    # List of input Parquet files along with their maximum rows and desired labels
    #input_files = [
    #    {'file': 'testdata/2405_clftest_benign_filtered.parquet', 'max_rows': 4000, 'label': 'benign'},
    #    {'file': 'testdata/2405_clftest_phishing_filtered.parquet', 'max_rows': 480, 'label': 'phishing'},
    #    {'file': 'testdata/2405_clftest_malware_filtered.parquet', 'max_rows': 292, 'label': 'malware'},
    #    {'file': 'testdata/dga_2310.parquet', 'max_rows': 300, 'label': 'dga'},
    #]
    
    #input_files = [
    #    {'file': 'testdata/2405_clftest_benign_filtered.parquet', 'max_rows': 300, 'label': 'benign'},
    #    {'file': 'testdata/2405_clftest_phishing_filtered.parquet', 'max_rows': 308, 'label': 'phishing'},
    #    {'file': 'testdata/2405_clftest_malware_filtered.parquet', 'max_rows': 292, 'label': 'malware'},
    #    {'file': 'testdata/dga_2310.parquet', 'max_rows': 300, 'label': 'dga'},
    #]

    input_files = [
        {'file': 'testdata/benign_2312_HTML.parquet', 'max_rows': 150, 'label': 'benign'},
        {'file': 'testdata/malware_2406_strict_HTML.parquet', 'max_rows': 150, 'label': 'malware'},
    ]

    # Number of rows to select in total
    #n_rows = 5072
    n_rows = 300
    
    # Read the first file to get the initial columns and create the first dataframe
    first_file_info = input_files[0]
    combined_df = pd.read_parquet(first_file_info['file'])
    
    # Limit the number of rows if necessary for the first file
    if len(combined_df) > first_file_info['max_rows']:
        combined_df = combined_df.sample(n=first_file_info['max_rows'], random_state=1)
    
    # Overwrite the "label" column with the specified label for the first file
    combined_df['label'] = first_file_info['label']
    
    # Get the columns from the first dataframe
    all_columns = combined_df.columns.tolist()

    # Process the remaining files
    for file_info in input_files[1:]:
        df = pd.read_parquet(file_info['file'])
        
        # Limit the number of rows if necessary
        if len(df) > file_info['max_rows']:
            df = df.sample(n=file_info['max_rows'], random_state=1)
        
        # Overwrite the "label" column with the specified label
        df['label'] = file_info['label']
        
        # Ensure all columns from the first dataframe are present
        for col in all_columns:
            if col not in df.columns:
                df[col] = None
        
        # Align the dataframe to the columns of the first dataframe
        df = df[all_columns]
        
        # Append the dataframe to the combined dataframe
        combined_df = pd.concat([combined_df, df], ignore_index=True)
    
    # Randomly select n_rows rows from the combined DataFrame
    selected_rows = combined_df.sample(n=n_rows, random_state=1)  # random_state for reproducibility

    selected_rows = selected_rows.drop(columns=['label_x', 'label_y'], errors='ignore')
    
    # Save the selected rows to a new Parquet file
    selected_rows.to_parquet('testdata/ver_html_test.parquet')
    print(df.columns)


Index(['domain_name', 'dns_has_dnskey', 'dns_A_count', 'dns_AAAA_count',
       'dns_MX_count', 'dns_NS_count', 'dns_TXT_count', 'dns_SOA_count',
       'dns_CNAME_count', 'dns_zone_level',
       ...
       'html_formjavascript', 'html_maliciousForm', 'html_mostCommon',
       'html_cssInternal', 'html_cssExternal', 'html_anchoresToContent',
       'html_anchoresToVoid', 'html_blankSpaces',
       'html_blocked_keywords_label', 'label'],
      dtype='object', length=266)


## Run classification

In [3]:
# Specify the parquet file with the dataset for classification
test_dataset = 'testdata/ver_html_test.parquet'

# Number of domain names to classify with each run of the pipeline (0 = classify all)
CHUNK_SIZE = 30

In [4]:
import pandas as pd
import numpy as np
import shap
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from classifiers.pipeline import Pipeline
from classifiers.options import PipelineOptions
#import matplotlib.pyplot as plt

# Initialize the classification pipeline
clf_options = PipelineOptions()
print(clf_options)
clf = Pipeline(clf_options)
print(clf)
# Read the input parquet file
input_df = pd.read_parquet(test_dataset)
print(input_df)
# Function to map labels to 'benign' or 'malign'
def map_label(label):
    if label == 'benign':
        return 'benign'
    else:
        return f'malign ({label})'

# Function to convert labels to binary classes
def binary_label(label):
    return 'negative' if label == 'benign' else 'positive'

# Apply label mapping
input_df['mapped_label'] = input_df['label'].apply(map_label)
input_df['binary_label'] = input_df['label'].apply(binary_label)

# Ensure SHAP JavaScript initialization
#shap.initjs()

# Determine the number of chunks
num_chunks = (len(input_df) + CHUNK_SIZE - 1) // CHUNK_SIZE if CHUNK_SIZE > 0 else 1

# Initialize counters for overall statistics
total_true_labels = []
total_pred_labels = []

# Format string for aligned output
header_format_str = "{:<3} | {:<50} | {:<18} | {:<10} | {:<10}"
data_format_str = "{:<3} | {:<50} | {:<18} | {:<10} | {:.6f}"

# Process the dataframe in chunks
for i in range(num_chunks):
    if CHUNK_SIZE > 0:
        start_idx = i * CHUNK_SIZE
        end_idx = start_idx + CHUNK_SIZE
        chunk_df = input_df[start_idx:end_idx]
    else:
        chunk_df = input_df

    # Perform your classification or processing on the working_df here
    print(f"===== Processing chunk {i+1}/{num_chunks} =====")

    chunk_without_label = chunk_df.drop(columns=['label', 'mapped_label', 'binary_label']) # Label should not be known to classifiers
    chunk_results = clf.classify_domains(chunk_without_label)

    # Collect predictions and true labels
    true_labels = chunk_df['binary_label'].values
    pred_labels = []
    for result in chunk_results:
        pred_label = 'negative' if result['aggregate_probability'] < 0.5 else 'positive'
        pred_labels.append(pred_label)

    # Update overall statistics
    total_true_labels.extend(true_labels)
    total_pred_labels.extend(pred_labels)

    # Display header for results
    print(header_format_str.format("Res", "Domain Name", "Actual Label", "Predicted", "Probability"))

    # Display results for each domain
    for idx, result in enumerate(chunk_results):
        actual_label = chunk_df.iloc[idx]['mapped_label']
        predicted_label = 'benign' if pred_labels[idx] == 'negative' else 'malign'
        domain_name = result['domain_name']
        aggregate_probability = result['aggregate_probability']
        status = "OK" if pred_labels[idx] == true_labels[idx] else "ER"
        print(data_format_str.format(status, domain_name[:50], actual_label, predicted_label, aggregate_probability))
        
        # Run debug_domain method for misclassified domains
        #if status == "ER":
            #print(f"Debugging misclassified domain: {domain_name}")
            #ndf_data = clf.pp.df_to_NDF(chunk_df, "phishing")  # Convert to NDF
            #debug_data = clf.clf_phishing_cnn.debug_domain(domain_name, ndf_data, chunk_df, n_top_features=10)
            
            # Print out the top n feature importances and values for each classifier
            #for classifier, data in debug_data.items():
            #    print(f"\nClassifier: {classifier}")
            #    print(f"Top {len(data['top_features'])} features for domain '{domain_name}':")
            #    for feature_info in data['top_features']:
            #        print(f"Feature: {feature_info['feature']}, Value: {feature_info['value']}, SHAP Value: {feature_info['shap_value']}")
                
                # Display the force plot for phishing_lgbm
                #if classifier == "phishing_cnn":
                #    base_value, shap_values, domain_row = data['force_plot_data']
                #    shap.force_plot(base_value, shap_values, domain_row)
                #    plt.show()

    # Calculate metrics for the current chunk
    accuracy = accuracy_score(true_labels, pred_labels)
    precision = precision_score(true_labels, pred_labels, pos_label='positive', average='binary')
    recall = recall_score(true_labels, pred_labels, pos_label='positive', average='binary')
    f1 = f1_score(true_labels, pred_labels, pos_label='positive', average='binary')

    tn, fp, fn, tp = confusion_matrix(true_labels, pred_labels, labels=['negative', 'positive']).ravel()
    false_positives = fp
    false_negatives = fn
    total_positives = tp + fp
    total_negatives = tn + fn

    fp_ratio = (false_positives / total_positives) if total_positives > 0 else 0
    fn_ratio = (false_negatives / total_negatives) if total_negatives > 0 else 0

    print(f"Chunk {i+1}/{num_chunks} metrics:")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print(f"False Positives: {false_positives} ({fp_ratio * 100:.2f}%)")
    print(f"False Negatives: {false_negatives} ({fn_ratio * 100:.2f}%)")
    print(f"===== Chunk {i+1}/{num_chunks} completed. =====")

# Calculate overall metrics
overall_accuracy = accuracy_score(total_true_labels, total_pred_labels)
overall_precision = precision_score(total_true_labels, total_pred_labels, pos_label='positive', average='binary')
overall_recall = recall_score(total_true_labels, total_pred_labels, pos_label='positive', average='binary')
overall_f1 = f1_score(total_true_labels, total_pred_labels, pos_label='positive', average='binary')

overall_tn, overall_fp, overall_fn, overall_tp = confusion_matrix(total_true_labels, total_pred_labels, labels=['negative', 'positive']).ravel()
overall_false_positives = overall_fp
overall_false_negatives = overall_fn
overall_total_positives = overall_tp + overall_fp
overall_total_negatives = overall_tn + overall_fn

overall_fp_ratio = (overall_false_positives / overall_total_positives) if overall_total_positives > 0 else 0
overall_fn_ratio = (overall_false_negatives / overall_total_negatives) if overall_total_negatives > 0 else 0

print("Overall metrics:")
print(f"Overall Accuracy: {overall_accuracy}")
print(f"Overall Precision: {overall_precision}")
print(f"Overall Recall: {overall_recall}")
print(f"Overall F1 Score: {overall_f1}")
print(f"Overall False Positives: {overall_false_positives} ({overall_fp_ratio * 100:.2f}%)")
print(f"Overall False Negatives: {overall_false_negatives} ({overall_fn_ratio * 100:.2f}%)")


/home/polonipeter/.cache/pypoetry/virtualenvs/classifiers-e0xTcLo8-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-09-17 17:01:54.692630: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


CNN model created
                       domain_name  dns_has_dnskey  dns_A_count  \
189             sultanroyal.com.tr             0.0            2   
123                    caldiae.gal             0.0            1   
185         gotowinawszystko.click             0.0            2   
213                   saldanha.top             0.0            2   
106                   hamster.desi             0.0            1   
..                             ...             ...          ...   
203         odoobravo.rawntech.com             0.0            0   
255        wiseguysbarandgrill.net             0.0            1   
72                step.npcj.edu.jm             0.0            2   
235  indigenousmediafoundation.org             0.0            2   
37               trusthouse.com.eg             0.0            2   

     dns_AAAA_count  dns_MX_count  dns_NS_count  dns_TXT_count  dns_SOA_count  \
189               2             2             2              1              1   
123            

## Optional: Generate preliminary results for training the final aggregation classifier

In [5]:
import pandas as pd
import numpy as np
from classifiers.pipeline import Pipeline

# Initialize the classification pipeline
clf = Pipeline()

# Read the input parquet file
input_df = pd.read_parquet(test_dataset)

input_df = input_df.sample(frac=1).reset_index(drop=True)

#preliminary_results_df = clf.generate_preliminary_results(input_df, output_file="test_preliminary_results.parquet")
preliminary_results_df = clf.generate_preliminary_results(input_df, add_final=False)

CNN model created


In [6]:
pd.reset_option('display.max_rows')
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 50)
pd.options.display.float_format = '{:.6f}'.format
preliminary_results_df

,domain_name,dns_available,dns_nonzero,tls_available,tls_nonzero,ip_available,ip_nonzero,rdap_available,rdap_nonzero,geo_available,geo_nonzero,phishing_cnn_result,phishing_lgbm_result,phishing_xgboost_result,phishing_deepnn_result,phishing_dns_nn_result,phishing_rdap_nn_result,phishing_geo_nn_result,phishing_ip_nn_result,phishing_html_lgbm_result,malware_html_lgbm_result,malware_lgbm_result,malware_xgboost_result,malware_deepnn_result,malware_dns_nn_result,malware_rdap_nn_result,malware_geo_nn_result,malware_ip_nn_result,dga_binary_deepnn_result,dga_binary_lgbm_result,phishing_sum,phishing_avg,phishing_prod,malware_sum,malware_avg,malware_prod,dga_binary_sum,dga_binary_avg,dga_binary_prod,total_sum,total_avg,total_prod,label
0,elotakarekossag.konnyuhitel.hu,0.650000,0.300000,0.833333,0.291667,1.000000,0.625000,0.833333,0.125000,1.000000,0.611111,0.000689,0.000391,0.000877,0.001714,0.016195,0.000692,0.043150,0.149577,0.000000,0.000000,0.000305,0.002458,0.000086,0.085861,0.001427,0.393644,0.566754,0.000001,0.000018,0.213283,0.023698,0.000000,1.050536,0.131317,0.000000,0.000019,0.000009,0.000000,1.263838,0.066518,0.000000,malware
1,wiseguysbarandgrill.net,0.950000,0.600000,0.833333,0.291667,1.000000,0.625000,1.000000,0.583333,1.000000,0.722222,0.389726,0.001994,0.004205,0.063423,0.027495,0.018503,0.390085,0.420532,0.004461,0.004461,0.467652,0.516943,0.198439,0.327826,0.048666,0.307506,0.365646,0.633339,0.030593,1.320424,0.146714,0.000000,2.237139,0.279642,0.000000,0.663932,0.331966,0.019376,4.221495,0.222184,0.000000,malware
2,aiswholesale.com,0.475000,0.000000,0.041667,0.000000,0.625000,0.000000,1.000000,0.291667,1.000000,0.000000,1.000000,0.002422,0.001522,0.004763,0.184629,0.028667,0.387495,0.414404,0.000000,0.000000,0.018182,0.026587,0.019657,0.394762,0.048591,0.604780,0.443381,0.011931,0.000499,2.023902,0.224878,0.000000,1.555941,0.194493,0.000000,0.012430,0.006215,0.000006,3.592273,0.189067,0.000000,malware
3,saldanha.top,1.000000,0.725000,0.833333,0.291667,1.000000,0.875000,1.000000,0.833333,1.000000,0.722222,0.202805,0.118992,0.051840,0.003039,0.020787,0.004914,0.521558,0.608665,0.744801,0.744801,0.990134,0.920979,0.676108,0.127129,0.401185,0.399883,0.521081,0.050233,0.000692,2.277400,0.253044,0.000000,4.781301,0.597663,0.004880,0.050925,0.025463,0.000035,7.109627,0.374191,0.000000,malware
4,mcandrewjewellery.com.au,1.000000,0.725000,0.833333,0.291667,1.000000,0.500000,0.875000,0.458333,1.000000,0.722222,0.059260,0.000638,0.001239,0.013375,0.020464,0.248514,0.595489,0.184353,0.000000,0.000000,0.043850,0.281584,0.277686,0.312844,0.072679,0.327624,0.279973,0.000003,0.000005,1.123333,0.124815,0.000000,1.596241,0.199530,0.000000,0.000008,0.000004,0.000000,2.719582,0.143136,0.000000,malware
5,marinecollagenelixir.com,0.900000,0.525000,0.833333,0.291667,1.000000,0.625000,1.000000,0.583333,1.000000,0.666667,0.985984,0.005044,0.039109,0.442562,0.220185,0.110148,0.094731,0.379587,0.000001,0.000001,0.825264,0.915739,0.746108,0.591719,0.193631,0.160225,0.422413,0.156435,0.001504,2.277349,0.253039,0.000000,3.855100,0.481888,0.000000,0.157940,0.078970,0.000235,6.290389,0.331073,0.000000,malware
6,tursaf.org.tr,1.000000,0.700000,0.833333,0.291667,1.000000,0.625000,0.958333,0.666667,1.000000,0.666667,0.007517,0.000224,0.000412,0.000377,0.013875,0.002220,0.172676,0.192140,0.000000,0.000000,0.005542,0.006804,0.002103,0.122376,0.010747,0.399883,0.409127,0.000000,0.000002,0.389443,0.043271,0.000000,0.956583,0.119573,0.000000,0.000003,0.000001,0.000000,1.346028,0.070844,0.000000,malware
7,www.nipponsoda.com,0.600000,0.225000,0.833333,0.291667,1.000000,0.625000,1.000000,0.708333,1.000000,0.722222,0.993279,0.704867,0.891511,0.997376,0.015623,0.100576,0.117976,0.078991,0.000000,0.000000,0.381572,0.348154,0.186243,0.026594,0.505025,0.328418,0.186704,0.000000,0.000020,3.900199,0.433355,0.000000,1.962710,0.245339,0.000000,0.000020,0.000010,0.000000,5.862929,0.308575,0.000000,malware
8,q0.wf,0.900000,0.550000,0.833333,0.291667,1.000000,0.625000,1.0

In [7]:
pd.set_option('display.max_rows', None)
preliminary_results_df[preliminary_results_df["label"].isin(["benign", "malware"])][["domain_name", "label", "phishing_cnn_result", "phishing_xgboost_result", "phishing_html_lgbm_result", "malware_html_lgbm_result","malware_lgbm_result"]]

,domain_name,label,phishing_cnn_result,phishing_xgboost_result,phishing_html_lgbm_result,malware_html_lgbm_result,malware_lgbm_result
0,elotakarekossag.konnyuhitel.hu,malware,0.000689,0.000877,0.000000,0.000000,0.000305
1,wiseguysbarandgrill.net,malware,0.389726,0.004205,0.004461,0.004461,0.467652
2,aiswholesale.com,malware,1.000000,0.001522,0.000000,0.000000,0.018182
3,saldanha.top,malware,0.202805,0.051840,0.744801,0.744801,0.990134
4,mcandrewjewellery.com.au,malware,0.059260,0.001239,0.000000,0.000000,0.043850
5,marinecollagenelixir.com,malware,0.985984,0.039109,0.000001,0.000001,0.825264
6,tursaf.org.tr,malware,0.007517,0.000412,0.000000,0.000000,0.005542
7,www.nipponsoda.com,malware,0.993279,0.891511,0.000000,0.000000,0.381572
8,q0.wf,malware,0.677601,0.001043,0.000000,0.000000,0.129314
9,vibra.dj,benign,0.000773,0.000503,0.000000,0.000000,0.000070


In [8]:
preliminary_results_df[["domain_name", "label", "malware_deepnn_result", "malware_lgbm_result", "malware_xgboost_result"]]

,domain_name,label,malware_deepnn_result,malware_lgbm_result,malware_xgboost_result
0,olkano.com,dga,0.001556,0.013970,0.047478
1,brandstorm.loreal.com,benign,0.000122,0.000514,0.002626
2,wytypowany-zwyciezca.com,malware,0.759562,0.383536,0.286979
3,oopt.fr,malware,0.122850,0.277118,0.445704
4,16chan.org,malware,0.627538,0.708378,0.589509
5,down.t0kbnpobket.biz,phishing,0.000658,0.010174,0.005187
6,bolsadetrabajosgt.com,phishing,0.000000,0.000067,0.002233
7,www.sprawozdania24.click,malware,0.849272,0.677257,0.907050
8,discord.writemall.top,phishing,0.000252,0.000411,0.002364
9,patasblancasvet.com,malware,0.794451,0.881981,0.884218


In [7]:
preliminary_results_df[preliminary_results_df["label"].isin(["benign", "malware"])][
    ["domain_name", "label", "malware_deepnn_result", "malware_lgbm_result", "malware_xgboost_result", "malware_dns_nn_result", "malware_rdap_nn_result", "malware_geo_nn_result"]
]


NameError: name 'preliminary_results_df' is not defined

In [10]:
preliminary_results_df[preliminary_results_df["label"].isin(["benign", "malware"])][
    ["domain_name", "label", "malware_deepnn_result", "malware_dns_nn_result", "malware_rdap_nn_result", "malware_geo_nn_result", "malware_ip_nn_result"]
]


,domain_name,label,malware_deepnn_result,malware_dns_nn_result,malware_rdap_nn_result,malware_geo_nn_result,malware_ip_nn_result
1,brandstorm.loreal.com,benign,0.000122,0.184353,0.001344,0.399883,0.548636
2,wytypowany-zwyciezca.com,malware,0.759562,0.775626,0.021642,0.399883,0.212463
3,oopt.fr,malware,0.122850,0.070304,0.024303,0.209001,0.450995
4,16chan.org,malware,0.627538,0.941982,0.049690,0.605018,0.279973
7,www.sprawozdania24.click,malware,0.849272,0.589228,0.398529,0.399883,0.452909
9,patasblancasvet.com,malware,0.794451,0.888655,0.061971,0.689959,0.330984
10,www.ipop.at,benign,0.000021,0.032429,0.175987,0.129151,0.006270
13,vurbont.xyz,malware,0.889243,0.398776,0.310197,0.399883,0.462252
14,srebny23.aleksytymon.com.pl,malware,0.000637,0.335020,0.000711,0.399883,0.637277
15,tools.pinpoll.com,benign,0.000008,0.076731,0.047117,0.016066,0.005857


In [11]:
#print(preliminary_results_df['malware_geo_nn_result'].unique())
#for value in preliminary_results_df['malware_geo_nn_result'].unique():
#    print(value)

In [12]:
preliminary_results_df[["label", "dga_binary_deepnn_result", "dga_binary_lgbm_result"]]

,label,dga_binary_deepnn_result,dga_binary_lgbm_result
0,dga,0.922759,0.195645
1,benign,0.000000,0.000002
2,malware,0.156716,0.007199
3,malware,0.136516,0.001036
4,malware,0.027879,0.001080
5,phishing,0.000001,0.001439
6,phishing,0.482309,0.006404
7,malware,0.000000,0.010450
8,phishing,0.000000,0.000006
9,malware,0.226647,0.003000


In [6]:
preliminary_results_df[["domain_name", "label", "phishing_cnn_result", "phishing_deepnn_result", "phishing_lgbm_result", "phishing_xgboost_result", "phishing_dns_nn_result", "phishing_rdap_nn_result"]]

NameError: name 'preliminary_results_df' is not defined

In [14]:
preliminary_results_df[["domain_name", "label", "malware_deepnn_result", "malware_lgbm_result", "malware_xgboost_result"]]

,domain_name,label,malware_deepnn_result,malware_lgbm_result,malware_xgboost_result
0,olkano.com,dga,0.001556,0.013970,0.047478
1,brandstorm.loreal.com,benign,0.000122,0.000514,0.002626
2,wytypowany-zwyciezca.com,malware,0.759562,0.383536,0.286979
3,oopt.fr,malware,0.122850,0.277118,0.445704
4,16chan.org,malware,0.627538,0.708378,0.589509
5,down.t0kbnpobket.biz,phishing,0.000658,0.010174,0.005187
6,bolsadetrabajosgt.com,phishing,0.000000,0.000067,0.002233
7,www.sprawozdania24.click,malware,0.849272,0.677257,0.907050
8,discord.writemall.top,phishing,0.000252,0.000411,0.002364
9,patasblancasvet.com,malware,0.794451,0.881981,0.884218


In [15]:
preliminary_results_df[["domain_name", "label", "badness_probability"]]

KeyError: "['badness_probability'] not in index"

In [ ]:
preliminary_results_df